## Import libraries

In [1]:
import os
import re
import json
import pandas as pd
import numpy as np
from pybis import Openbis
import warnings
import string
warnings.filterwarnings("ignore")

## Functions

In [2]:
def load_json(filepath: str) -> dict:
    return json.load(open(filepath, "r"))

def log_in(bisurl='openbis', bisuser='admin', bispasswd='changeit'):
    """Function to login to openBIS."""
    if Openbis(bisurl, verify_certificates=False).is_token_valid():
        session = Openbis(bisurl, verify_certificates=False)
    else:
        Openbis(bisurl, verify_certificates=False).login(bisuser, bispasswd, save_token=True)
        session = Openbis(bisurl, verify_certificates=False)
    return session

## Connect to openBIS

In [3]:
session = log_in(bisurl='openbis', bisuser='admin', bispasswd='123456789')

## Load all object schemas

In [4]:
objects_schemas_foldername = "New_Metadata_Schemas"
all_objects_schemas_filenames = [f"{objects_schemas_foldername}/{object_schema_filename}" for object_schema_filename in os.listdir(objects_schemas_foldername)]

## Create property types, vocabularies, and object types

In [7]:
for object_schema_filename in all_objects_schemas_filenames:
    object_schema_file = load_json(object_schema_filename)
    
    # Create property types and vocabularies
    for property_code, property_settings in object_schema_file["properties"].items():
        
        # In openBIS, the property type "name" is already created and can be used to find objects in the GUI. Therefore, it should be used instead of creating a new one.
        if property_code == "name":
            property_code = "$name"
            
        if "openBISType" in property_settings:
            property_type_dict = {
                "code": property_code,
                "label": property_settings["title"],
                "description": property_settings["description"],
                "dataType": property_settings["openBISType"],
                "managedInternally": False,
            }
            
            if property_settings["openBISType"] == "CONTROLLEDVOCABULARY":
                
                vocabulary_dict = {
                    "code": f"{property_code}Vocabulary",
                    "description": f"{property_settings['title']} vocabulary",
                    "terms": []
                }
                
                if "enum" in property_settings:
                    for term in property_settings["enum"]:
                        term_dict = {
                            "code": term.replace(" ", ""),
                            "description": term,
                            "label": term
                        }
                        
                        vocabulary_dict["terms"].append(term_dict)
                else:
                    # Case when the terms are just the alphabetic letters
                    for term in list(string.ascii_lowercase):
                        term_dict = {
                            "code": term,
                            "description": term,
                            "label": term
                        }
                        
                        vocabulary_dict["terms"].append(term_dict)
                        
                try:
                    session.new_vocabulary(**vocabulary_dict).save()
                except ValueError:
                    print(f"{vocabulary_dict['code']} already exists.")
                
                property_type_dict["vocabulary"] = vocabulary_dict["code"]
            
            if "XML" in property_settings["openBISType"]:
                match = re.match(r"(\w+)\s\(([\w\s]+)\)", property_settings["openBISType"])
                property_type = match.group(1)
                custom_widget = match.group(2)
                property_type_dict["dataType"] = property_type
                property_type_dict["metaData"] = {"custom_widget": custom_widget}
            
            if property_settings["openBISType"] == "MULTILINE_VARCHAR":
                property_type_dict["metaData"] = {"custom_widget": "Word Processor"}
                
            try:
                session.new_property_type(**property_type_dict).save()
            except ValueError:
                print(f"{property_type_dict['code']} already exists.")

    # Create object types

    object_type_dict = {
        "code": object_schema_file["title"].replace(" ", "_"),
        "generatedCodePrefix": object_schema_file["openBISCode"]
    }

    try:
        object_type = session.get_object_type(object_type_dict["code"])
    except ValueError:
        print(f"{object_type_dict['code']} does not exist.")
        object_type = session.new_object_type(autoGeneratedCode=True, 
                                                subcodeUnique=False,
                                                listable=True,
                                                showContainer=False,
                                                showParents=True,
                                                showParentMetadata=False,
                                                **object_type_dict).save()

    for property_code in object_schema_file["properties"]:
        if property_code == "name":
            property_code = "$name"
        try:
            object_type.assign_property(session.get_property_type(code=property_code))
        except ValueError:
            print(f"{property_code} does not exist.")
    
    settings_sample = session.get_sample("/ELN_SETTINGS/GENERAL_ELN_SETTINGS")
    settings = json.loads(settings_sample.props["$eln_settings"])
    settings['sampleTypeDefinitionsExtension'][object_type.code] = {'SAMPLE_PARENTS_DISABLED': False,
                                                                    'SAMPLE_PARENTS_ANY_TYPE_DISABLED': False,
                                                                    'SAMPLE_CHILDREN_DISABLED': False,
                                                                    'SAMPLE_CHILDREN_ANY_TYPE_DISABLED': False,
                                                                    'USE_AS_PROTOCOL': False,
                                                                    'ENABLE_STORAGE': False,
                                                                    'SHOW': True, 
                                                                    'SHOW_ON_NAV': True}

    settings_sample.props['$eln_settings'] = json.dumps(settings)
    settings_sample.save()

{"method": "createPropertyTypes", "params": ["admin-240606103043861xFA8EB3804354A48BF6E747A9A0DB2FA4", [{"@type": "as.dto.property.create.PropertyTypeCreation", "code": "$name", "label": "Name", "description": "The name of the dosing.", "managedInternally": false, "dataType": "VARCHAR", "vocabularyId": null, "materialTypeId": null, "schema": null, "transformation": null, "metaData": null}]], "id": "2", "jsonrpc": "2.0"}
$name already exists.
{"method": "createPropertyTypes", "params": ["admin-240606103043861xFA8EB3804354A48BF6E747A9A0DB2FA4", [{"@type": "as.dto.property.create.PropertyTypeCreation", "code": "molecule_name", "label": "Molecule name", "description": "The name of the used gas molecule.", "managedInternally": false, "dataType": "VARCHAR", "vocabularyId": null, "materialTypeId": null, "schema": null, "transformation": null, "metaData": null}]], "id": "2", "jsonrpc": "2.0"}
molecule_name already exists.
{"method": "createPropertyTypes", "params": ["admin-240606103043861xFA8E

In [ ]:
settings_sample = session.get_sample("/ELN_SETTINGS/GENERAL_ELN_SETTINGS")
settings = json.loads(settings_sample.props["$eln_settings"])
settings['sampleTypeDefinitionsExtension'][st.code] = {'SAMPLE_PARENTS_DISABLED': False,
                                                                 'SAMPLE_PARENTS_ANY_TYPE_DISABLED': False,
                                                                 'SAMPLE_CHILDREN_DISABLED': False,
                                                                 'SAMPLE_CHILDREN_ANY_TYPE_DISABLED': False,
                                                                 'USE_AS_PROTOCOL': False,
                                                                 'ENABLE_STORAGE': False,
                                                                 'SHOW': True, 
                                                                 'SHOW_ON_NAV': True}

settings_sample.props['$eln_settings'] = json.dumps(settings)
settings_sample.save()

In [ ]:
settings_sample = session.get_sample("/ELN_SETTINGS/GENERAL_ELN_SETTINGS")
settings = json.loads(settings_sample.props["$eln_settings"])
# settings['sampleTypeDefinitionsExtension'] = {}

# settings['sampleTypeDefinitionsExtension'][object_type.code] = "{'SAMPLE_PARENTS_DISABLED': False, 'SAMPLE_PARENTS_ANY_TYPE_DISABLED': False, 'SAMPLE_CHILDREN_DISABLED': False, 'SAMPLE_CHILDREN_ANY_TYPE_DISABLED': False, 'USE_AS_PROTOCOL': False, 'ENABLE_STORAGE': False, 'SHOW': True, 'SHOW_ON_NAV': True}"

# settings_sample.props['$eln_settings'] = settings
# settings_sample.save()